# Assignment 1

*Part of the course:
Machine Learning (code: INFOB3ML), fall 2022, Utrecht University*

Total points: 10

Deadline: Friday 16 September, 23:59

**Write your names and student numbers here: ___**

Submit one ipynb file per pair.

**Before you submit, click Kernel > Restart & Run All to make sure you submit a working version of your code!**

## Linear Regression with Regularisation
In this assignment, you will perform several simulation experiments with linear regression, in order to investigate the effects of regularisation.

Those who followed the course Introduction to Machine Learning will see that this assignment resembles the linear regression assignment from that course. If you didn't take that course, don't worry: everything you need to know about it is also in chapter 1 of the Rogers & Girolami book.

## Overview
### Data generation
Every datapoint $(x,t)$ will be sampled randomly, with both $x$ and $t$ in $\mathbb{R}$. (Note that the book from Introduction to Machine Learning called the output $y$, while this course's book calls it $t$.) The input $x$ is uniformly distributed between $-1$ and $1$.

Once we have our input $x$, we generate our output $t$ according to $t = 1 - \cos(x) + \epsilon$, where $\epsilon$ is normally distributed with expected value = 0 and variance = $\sigma^2$.
All the random numbers should be generated **independently** from one another.

### Regression

We'll implement *regularised* regression, adding a penalty $\lambda \mathbf{w}^T \mathbf{w}$ to our training loss. Linear regression with this form of regularisation penalty is also called *ridge regression*. We are going to try out different values of $\lambda$.

We'll use regression with order five polynomials like in the book. This means that for each weight vector $\mathbf{w}$, our hypothesis is of the form
$$f(x; \mathbf{w}) = \sum_{i=0}^5 w_i x^i.$$

## Your Code

To make it clear what your code is supposed to do and how it should be formatted, we provide you general schema for each to-be-written function. Some functions come with additional hints about useful in-built functions or procedural details. You might write the function's body differently than the hints suggests. That's totally fine as long as the function works as it supposed to work.

Use numpy for functionalities involving vectors and matrices. [Here is a handy guide (in notebook form) that deals with numpy arrays, matrices and number generation.](https://github.com/ageron/handson-ml/blob/master/tools_numpy.ipynb)

### Data generation with noise

**Assignment 1** (1 point)

Write a function `generate_data` you can use to generate a dataset and outputs the pair of vectors `(x,t)`, accepting parameters $N$ and $\sigma^2$. Be sure to check if your normal-distribution-generator needs $\sigma$ (standard deviation) or $\sigma^2$ (variance) as input parameter. Both `x` and `t` should be 1-dimensional numpy arrays, i.e. their shape should be `(N,)`.

In [1]:
import numpy as np
import math
# Some functions you may find useful (here and later):
# np.cos, np.ones, np.random.random_sample, np.random.randn,
# np.linalg.inv, np.hstack, np.matmul (or @), np.dot, math.sqrt.

def generate_data(N, sigma_squared):
    x_vector = (np.random.random_sample((N,)) - 0.5) * 2
    t_vector = []
    index = 0
    sigma = math.sqrt(sigma_squared)
    for index in range(N):
        error = np.random.normal(0.0, sigma)
        t_vector.append(1 - np.cos(x_vector[index]) + error)

        
    return x_vector, np.asarray(t_vector)
    
        
        

In [2]:
# ██████████ TEST ██████████
# (These "TEST" blocks can help you quickly check if there's something obviously wrong with the code you wrote.)
# Setting a seed helps to make the data generation deterministic for comparison reasons
np.random.seed(0)
toy_xs, toy_t = generate_data(3, 0.1)
# Check if the shapes of the output arrays are as specified above:
print(toy_xs.shape)
print(toy_xs)
print(toy_t.shape)
print(toy_t)

(3,)
[0.09762701 0.43037873 0.20552675]
(3,)
[-0.71254663  0.51289628 -0.24544636]


Next, in order to learn 5th-order polynomials, we want to have a function that computes the matrix $\mathbf{X}$. (See equation (1.18) on page 28 of the book for what this matrix should look like.)

**Question 2** (1 point)

(a) When dealing with $N$ data points, what should the size of the $\mathbf{X}$-matrix be? (Give the number or rows and the number of columns.)

*With 5'th order polynomials, the matrix X should be 6xN. 6 columns, N rows.*

(b) Where in the matrix will you find the vector `x_scalars`?

*The x_scalars will be present in the 2nd column.*

(c) Where in the matrix will you find $\mathbf{x}_1$ (the feature vector associated to the first data point)?

*The first datapoint will be in the first row.*

**Assignment 3** (1 point)

Write a function `compute_X_matrix` that takes a numpy array `x_scalars` as produced by your code above, and returns the matrix `X` needed by linear regression with 5th-order polynomials.

In [4]:
def compute_X_matrix(x_scalars):
    x_matrix = []
    N = len(x_scalars)
    for index in range(N):
        x_scalar = x_scalars[index]
        partial_x = [1, x_scalar, x_scalar**2, x_scalar**3,  x_scalar**4,  x_scalar**5] #Miss een beetje bagger dit maar werkt allee met 5th
        x_matrix.append(partial_x)
        
    return np.asmatrix(x_matrix)
        
    

In [5]:
# ██████████ TEST ██████████
toy_X = compute_X_matrix(toy_xs)
print(toy_xs)
print(toy_X)
print(toy_X.shape)

[0.09762701 0.43037873 0.20552675]
[[1.00000000e+00 9.76270079e-02 9.53103266e-03 9.30486201e-04
  9.08405836e-05 8.86849437e-06]
 [1.00000000e+00 4.30378733e-01 1.85225854e-01 7.97172681e-02
  3.43086168e-02 1.47656990e-02]
 [1.00000000e+00 2.05526752e-01 4.22412458e-02 8.68170607e-03
  1.78432285e-03 3.66726080e-04]]
(3, 6)


### Fitting linear regression

**Assignment 4** (1 point)

Write code that fits a regularised linear regression hypothesis to training data, in other words, a function that computes our $\hat{\mathbf{w}}$. Use numpy to carry out the necessary matrix operations to find an analytic solution; don't use linear regression functionality from Python packages for machine learning. In other words, compute $\hat{\mathbf{w}}$ according to the equation (1.21) on page 36 of the book. Give your function a parameter `lamb` which tells it the value of $\lambda$.
   

In [41]:
def fit_ridge(X, t, lamb):
    N = len(X)
    w_hat = []
    XTX = X.T @ X
    XTXNlambI_1 = np.linalg.inv(XTX + N * lamb * np.identity(6))
    XTt = X.T @ t
    print("XT", X.T.shape)
    print("t", t.shape)
    print("XTt", (X.T @ t).shape) # XTt is dus ergens dubbel ge-transposed, want hij is precies het omgekeerde van wat ie moet zijn
    # In de print messages staat dus: we hebben XT en t, die zijn 6,3 en 3,1 en XTt is 1,6 HUH
    total = XTXNlambI_1 @ XTt.T # Ik heb hem hier maar even terug getransposed zodat we doorkunnen maar eigenlijk is dit kut
    
    return total # w returnt nu een 6x1 matrix, is dat de bedoeling? anders doen we iets fundamenteel verkeerd


In [16]:
# ██████████ TEST ██████████
# this test uses values for toy_X and toy_t printed above
# You migh need to define these values by hand
# if your "generate_data" returns different data
print(toy_X)
print(toy_t)
print(fit_ridge(toy_X, toy_t, 0.01))
print(fit_ridge(toy_X, toy_t, 100))

[[1.00000000e+00 9.76270079e-02 9.53103266e-03 9.30486201e-04
  9.08405836e-05 8.86849437e-06]
 [1.00000000e+00 4.30378733e-01 1.85225854e-01 7.97172681e-02
  3.43086168e-02 1.47656990e-02]
 [1.00000000e+00 2.05526752e-01 4.22412458e-02 8.68170607e-03
  1.78432285e-03 3.66726080e-04]]
[-0.71254663  0.51289628 -0.24544636]
XT (6, 3)
t (3,)
XTt (1, 6)
[[-0.69726571]
 [ 1.88156984]
 [ 1.00802551]
 [ 0.45015441]
 [ 0.19470722]
 [ 0.08376452]]
XT (6, 3)
t (3,)
XTt (1, 6)
[[-1.47003611e-03]
 [ 3.38997111e-04]
 [ 2.60495694e-04]
 [ 1.27356278e-04]
 [ 5.71334367e-05]
 [ 2.49870604e-05]]


### Squared loss over the data

The following code (which you don't need to change) evaluates a learned linear regression function $\hat{\mathbf{w}}$ with respect to the data $\mathbf{X}, \mathbf{t}$ using the squared error loss. This Python function can be used to compute training, validation or test loss for $\hat{\mathbf{w}}$, depending of the kind of data passed to it.

In [37]:
def compute_loss(w, X, t):
    N, k = X.shape
    t_hat = X @ w
    t_error = t_hat - t
    sum_of_squared_errors = t_error.T @ t_error
    loss = sum_of_squared_errors / N
    return loss

## Visualising regularisation

Now it's time to do some experiments and look at the results.

First, the function provided below will plot the target function $1 - \cos(x)$, a regression function, and the training data all in one plot. 

In [18]:
import matplotlib.pyplot as plt

def plot_regression_result(xs_train, t_train, w, include_target_function=True):
    xs_plot = np.linspace(-2, 2, 101)
    X_plot = compute_X_matrix(xs_plot)
    if include_target_function:
        plt.plot(xs_plot, 1 - np.cos(xs_plot), c='black') # target 
    plt.scatter(xs_train, t_train, c='blue', marker=".")
    plt.plot(xs_plot, X_plot @ w, c='red')
    plt.ylim(-1, 1.5)
    plt.show()

**Assignment 5** (1 point)

Generate a dataset of $N=15$ data points with noise level $\sigma^2 = 0.1$. (You'll use this dataset in assignments/questions 5 through 7.) For the values of $\lambda$ provided in the code below, fit a regularised regression curve to the data and compute the loss. Display the results in a plot.

In [40]:
lambdas = 10 ** np.linspace(-5, 5, 11)
losses = np.zeros_like(lambdas) # Loss is blijkbaar een matrix, dus dit wordt een lijst met matrices
# probleem is, hier wordt een lijst gemaakt waar ints in moeten, maar loss is een matrix. Waarom output loss een matrix?
# klopt dat wel? anders is onze input stuk want de compute_loss functie was al geschreven 

np.random.seed(0)

# Your code here
x_vector, t_vector = generate_data(15, 0.1)
x_matrix = compute_X_matrix(x_vector)

for index, lamb in enumerate(lambdas):
    weights = fit_ridge(x_matrix, t_vector, lamb)
    loss = compute_loss(weights, x_matrix, t_vector)
    losses[index] = loss # dit is niet legaal blijkbaar
    
    
plt.semilogx(lambdas, losses, c='black') # target 
plt.show()

XT (6, 15)
t (15,)
XTt (1, 6)


ValueError: setting an array element with a sequence.

**Question 6** (1 point) Your plot should show that as $\lambda$ gets larger, the loss also gets larger. Explain why this is to be expected.

*Your answer here*

**Assignment/Question 7** (1 point)

Using the same data, answer the following questions:

For what values of $\lambda > 0$ do you clearly see overfitting? For what values of $\lambda$ do you see underfitting? To support your answer, include plots for some values of $\lambda$, and point out what features of those plots tell you that over-/underfitting is going on.

In [ ]:
# Your code here

*Your answer here*

## Cross-validation

To find a good value of $\lambda$, a variety of techniques exist. One that obviously does *not* work is to look at the training loss as a function of $\lambda$ (like you plotted above): that would always suggest to make $\lambda$ as small as possible! A versatile technique that you've already seen in an earlier course (or in section 1.5 of the book) is **cross-validation**.

**Assignment 8** (1 point)

Write some code to do the following: sample a new dataset of $N = 50$ data points and $\sigma^2=0.1$. (You'll use this dataset for all the remaining assignments and questions.) Write a function that, given data and value of $\lambda$, computes the leave-one-out cross-validation (LOOCV) loss, as explained in section 1.5.2 of the book. Then make a plot similar to what we did in assignment 5 for the training loss, but this time displaying the LOOCV loss as a function of $\lambda$.

Note that the third argument, `fitting_function`, should be the name of a function that `LOOCV` can call to compute `w`. If `fit_ridge` is passed, your previously written function will be used. But later, you'll call it with a different fitting function.

In [ ]:
def LOOCV(X, t, fitting_function, lamb):
    N, k = X.shape
    sum_of_losses = 0.0
    for leave_out in range(N):
        # Your code here
        pass
        
    return sum_of_losses / N

# Your code here to sample a larger dataset, and to make the plot of the LOOCV loss for each lambda

**Question 9** (0.5 points): What value of $\lambda$ does LOOCV point you to? Look at a plot of the resulting regression function. Does it look reasonable?

*Your answer here*

## Lasso regression

As the book mentions, when doing regularisation, using the squares of $\mathbf{w}$ as a penalty is just one of many possibilities. It has the advantage of having an analytical solution. But other options exist that may have other advantages, and while they may not be analytically computable, still there exist efficient algorithms for working with them. A particularly popular one is to use the sum of absolute values of $\mathbf{w}$ as a penalty: we will find the $\mathbf{w}$ that minimizes
$$\mathcal{L}' = \mathcal{L} + \lambda \sum_i \lvert w_i \rvert.$$
This is called the 'lasso' (which is an acronym for 'least absolute shrinkage and selection operator', but of course most people just remember the acronym).

There is no direct formula for computing the $\mathbf{w}$ that minimizes $\mathcal{L}'$. The next alternative would be to use (stochastic) gradient descent. Unfortunately, that also doesn't work very nicely here, because as a function of $\mathbf{w}$, $\mathcal{L}'$ is not differentiable wherever $\mathbf{w}$ has at least one entry equal to zero. But variants of gradient descent have been developed that can deal with this problem (such as [proximal gradient descent](https://en.wikipedia.org/wiki/Proximal_gradient_method)), and implementations are readily available. The fitting function provided below uses such an implementation.

In [ ]:
from sklearn.linear_model import Lasso

def fit_lasso(X, t, lamb):
    clf = Lasso(lamb, fit_intercept=False, max_iter=100000)
    clf.fit(X, t)
    return clf.coef_

**Assignment 10** (0.5 point) Again plot the LOOCV losses as a function of $\lambda$, but this time for lasso regression instead of ridge regression. Read off the values of $\lambda$ that minimizes the LOOCV loss, and display the regression function for that $\lambda$ in a separate plot.

In [ ]:
# Your code here

An important property of lasso regularisation is its tendency to make some weigths exactly equal to 0. (Well, mathematically that's true, but you should never rely on things being *exactly* equal when a numerical algorithm is involved. Instead, check whether the difference between them is very small, say less than `1e-9`.)

**Assignment 11** (0.5 points)

What is the smallest $\lambda$ in `lambdas` for which you observe this happening for some $w_i$? For that $\lambda$, make a plot where $w_i$ varies along the horizontal axis. On the vertical axis, plot the regularised loss $\mathcal{L}'$ of the weight vector, with all entries other than $w_i$ kept equal to the optimal lasso solution. Choose the range of $w_i$-values small enough that you see a nondifferentiability in the graph.

In [ ]:
# Your code here

**Question 12** (0.5 points): Use this graph to explain why lasso regression has a tendency to make some weights equal to 0.

*Your answer here*

    
---

<br>
<br>

**Remember: Before you submit, click Kernel > Restart & Run All to make sure you submit a working version of your code!**